In [2]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.schema import (
    HumanMessage,
    SystemMessage
)

llm = ChatOpenAI(openai_api_key='', temperature=0.9, model_name="gpt-4")

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
response = llm([HumanMessage(content="Qual a capital do Brasil?")])
response

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='A capital do Brasil é Brasília.')

In [4]:
import os
import pandas as pd
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%pip install textract

Note: you may need to restart the kernel to use updated packages.


In [6]:

import textract
doc = textract.process("/Users/lucastavrs2/Desktop/Lucas/langchainCourse/ainda.pdf")

with open("/Users/lucastavrs2/Desktop/Lucas/langchainCourse/ainda.txt", 'w', encoding="utf-8") as f:
    f.write(doc.decode('utf-8'))


with open("/Users/lucastavrs2/Desktop/Lucas/langchainCourse/ainda.txt", "r", encoding="utf-8") as f:
    text= f.read()


tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 512,
    chunk_overlap= 24,
    length_function= count_tokens,
)

chunks = text_splitter.create_documents([text])

In [7]:
%pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [8]:
embeddings = OpenAIEmbeddings(openai_api_key="", model="text-embedding-ada-002")

db = FAISS.from_documents(chunks, embeddings)

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
query = "Fale sobre a introdução"
docs = db.similarity_search(query) 
docs[0]

Document(page_content='Dedicatória \nÀs filhas mais novas dos autores: a Sofia e a Diana! \n \nRicardo Queirós e Filipe Portela \n \n \n \n \nAgradecimentos \nEm primeiro lugar, gostaríamos de enaltecer o trabalho de \nqualidade das revisões feitas pela Ana Correia e Laura Faia, da \nFCA.  \n\nGostaríamos também de agradecer à equipa ioTeam, \nnomeadamente à Bruna Ferreira pelo desenho do logotipo e ao \nDaniel Carneiro pelo apoio no desenvolvimento da API \nincluída no projeto apresentado neste livro. \n\nFinalmente, um agradecimento à Sandra Correia e ao  \nEng.° Frederico Annes, da FCA, pela simpatia e pelo  \napoio no decorrer desta nossa aventura editorial. \n \nRicardo Queirós e Filipe Portela \n \n\n\x0cTITULO DO LIVRO \n\n \n\n© FCA – Editora de Informática \n\n\x0c \n\nÍÍNNDDIICCEE  \n\nOS AUTORES ......................................................................................................XI \nPREFÁCIO...................................................................

In [11]:
chain = load_qa_chain(OpenAI(openai_api_key="", temperature=0),chain_type="stuff")

chain.run(input_documents=docs, question=query)

/Users/lucastavrs2/miniconda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' A introdução deste livro apresenta aos leitores as principais tendências no desenvolvimento Web, fornecendo um roteiro simples e prático do estado atual da Web em termos de desenvolvimento front-end e back-end. Além disso, é introduzido o projeto prático Animalec, uma aplicação educacional sobre animais, e são abordadas as várias fases de gestão do projeto, como análise de requisitos, prototipagem e edição e controlo de versões do código da aplicação.'